In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from distributions.distribution_by_sequence import DistributionBySequence
from distributions.sequence import Sequence
from algorithms.semi_bandit_exp3 import SemiBanditExp3
from experiment_manager.experiment_manager import ExperimentManager

In [ ]:
exp_manager = ExperimentManager()
algos = [SemiBanditExp3]

length = 2000
d = 2
K = 2
actionset = [np.ones(K, dtype=bool)]
for i in range(K):
    buffer = np.zeros(K, dtype=bool)
    buffer[i] = 1
    actionset.append(buffer)
actionset = np.array(actionset)

seq = Sequence(actionset, length, d)
dists = [DistributionBySequence(seq)]